In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sn
import joblib

In [2]:
df = pd.read_excel('C:/Users/dell/Documents/tenserflow/Cow Health Predictor/cattle_dataset.xlsx')

In [3]:
def change(x):
    if x == 'Black faece':
        x = 'Black faeces'
    elif x == 'Fresh blood in faece':
        x = 'Fresh blood in faeces'
    elif x == 'very liquid faece':
        x = 'very liquid faeces'
    return x

faecal_consistency_update = df['faecal_consistency']
t = faecal_consistency_update.map(change)
df['faecal_consistency'] = t

In [4]:
df['body_condition_score'] = df['body_condition_score'].astype(str)

In [5]:
dummies = pd.get_dummies(df[['breed_type', 'body_condition_score', 'faecal_consistency']] , dtype='int') 

In [6]:
df = pd.concat([df , dummies] ,axis=1)

In [7]:
df.drop(['breed_type', 'body_condition_score', 'faecal_consistency'] , axis=1 , inplace=True)

In [8]:
def partition(x):
    if x == 'healthy':
        return 1
    return 0

health_update = df['health_status']
t = health_update.map(partition)
df['health_status'] = t

In [9]:
y = df['health_status']
X = df.drop('health_status', axis=1)

In [10]:
X

,body_temperature,milk_production,respiratory_rate,walking_capacity,sleeping_duration,heart_rate,eating_duration,lying_down_duration,ruminating,rumen_fill,...,body_condition_score_1,body_condition_score_2,body_condition_score_3,body_condition_score_4,body_condition_score_5,faecal_consistency_Black faeces,faecal_consistency_Fresh blood in faeces,faecal_consistency_extremely firm,faecal_consistency_ideal,faecal_consistency_very liquid faeces
0,38.2,13.6,36,12432,3.5,50,3.2,15.0,6.0,3,...,0,0,1,0,0,0,0,1,0,0
1,38.9,24.1,37,11987,4.2,62,3.9,12.2,5.8,4,...,0,1,0,0,0,0,0,0,1,0
2,38.6,14.7,42,13121,3.2,71,3.0,12.6,5.9,2,...,0,0,1,0,0,0,0,0,1,0
3,39.5,26.2,29,12055,3.8,68,3.6,13.5,6.0,2,...,0,0,0,1,0,1,0,0,0,0
4,39.7,22.5,48,10352,4.6,63,3.1,13.5,6.3,5,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,39.9,6.8,28,6942,7.1,47,1.9,15.0,3.1,3,...,0,0,0,1,0,0,1,0,0,0
174,40.2,7.2,20,6652,6.5,41,2.0,15.5,3.6,4,...,0,0,0,1,0,1,0,0,0,0
175,39.8,7.7,16,6705,6.6,37,2.6,14.8,4.2,3,...,0,0,1,0,0,0,0,1,0,0
176,39.5,5.7,21,6272,6.8,45,2.2,15.0,3.1,2,...,0,0,1,0,0,0,0,0,0,1


In [11]:
X.to_excel('X_final.xlsx')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=None)

In [13]:
X_train.to_excel('X_train.xlsx')
X_test.to_excel('X_test.xlsx')

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X_train.shape[1],)),
    keras.layers.Dense(22, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(11, activation='relu'),
    keras.layers.Dense(len(y.unique()), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.fit(X_train, y_train, epochs=250)

Epoch 1/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1909 - loss: 0.9045
Epoch 2/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2406 - loss: 0.8393 
Epoch 3/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2850 - loss: 0.7714 
Epoch 4/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3517 - loss: 0.7156 
Epoch 5/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5066 - loss: 0.6479 
Epoch 6/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6156 - loss: 0.6018 
Epoch 7/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7567 - loss: 0.5611 
Epoch 8/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8381 - loss: 0.5311 
Epoch 9/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9504 - loss: 0.4631 
Epoch 10/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9769 - loss: 0.4234 
Epoch 11/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9853 - loss: 0.3933 
Epoch 12/250
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9916 - loss

In [17]:
joblib.dump(model, 'Cow Health Predictor Final.pkl')

['Cow Health Predictor Final.pkl']